# Backtesting (live)

In [1]:
import keras

import sys

sys.path.append('.')

import os
import datetime
import numpy as np
import pandas as pd

from datetime import datetime

np.random.seed(7)

from mikasa.strategy import Strategy
from mikasa.event import SignalEvent, OrderEvent
from mikasa.backtest import Backtest
from mikasa.data import PoloniexDataHandler
from mikasa.portfolio import NaivePortfolio
from mikasa.execution import SimulatedExecutionHandler

from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [5]:
class PredictStrategy(Strategy):
    def __init__(self, bars, events):
        self.bars = bars
        self.symbol_list = self.bars.symbol_list
        self.events = events

        self.lag = 12

        csv_dir = './datasets/5min/'

        df = pd.read_csv(os.path.join(csv_dir, symbol_list[0] + '_test.csv'))
        for lag in range(1, self.lag + 1):
            df['close-' + str(lag)] = df['close'] - df.shift(lag)['close']
        df.dropna(inplace=True)

        self.close_fields = ['close-{}'.format(i + 1) for i in range(self.lag)]

        df['up'] = df['close'] < df.shift(-1)['close']

        self.scaler = StandardScaler()
        X = df[self.close_fields].values
        self.scaler.fit(X)

        self.model = keras.models.load_model('model_5min_btc_eth.h5')

    def calculate_signals(self, event):
        if event.type == 'MARKET':
            dt = self.bars.get_latest_bars_values(self.symbol_list[0], 'datetime')[0]

            X = self.bars.get_latest_bars_values(self.symbol_list[0], 'close', N=self.lag + 1)
            if len(X) == self.lag + 1:
                df = pd.DataFrame(X, columns=['close'])
                for lag in range(1, self.lag + 1):
                    df['close-' + str(lag)] = df['close'] - df.shift(lag)['close']
                df.dropna(inplace=True)
                X = df[self.close_fields].values
                X = self.scaler.transform(X)
                prediction = self.model.predict(X)[0]
                if prediction[1] > 0.7:
                    signal = SignalEvent(1, self.symbol_list[0], dt, 'LONG', 1.0)
                    self.events.put(signal)
                if prediction[0] > 0.7:
                    signal = SignalEvent(1, self.symbol_list[0], dt, 'EXIT', 1.0)
                    self.events.put(signal)

In [ ]:
#class PoloniexDataHandler(HistoricCSVDataHandler):
#    def __init__(self, events, symbol_list, period):
#        self.events = events
#        self.symbol_list = symbol_list
#
#        self.latest_symbol_data = {s: [] for s in self.symbol_list}
#        self.continue_backtest = True
#
#        self.period = period
#
#    def _get_new_bar(self, symbol):
#        polo = Poloniex()
#        start = datetime.datetime.now() - datetime.timedelta(days=1)
#        data = polo.returnChartData(symbol, period=self.period, start=start.timestamp())[-1]
#
#        yield DataPoint({
#            'symbol': symbol,
#            'datetime': datetime.datetime.fromtimestamp(data['date']),
#            'timestamp': data['date'],
#            'open': data['open'],
#            'high': data['high'],
#            'low': data['low'],
#            'close': data['close'],
#            'volume': data['volume']
#        })

In [6]:
class PoloniexBacktest(Backtest):
    def _generate_trading_instances(self):
        self.data_handler = self.data_handler_cls(self.events,
                                                  self.symbol_list, period=300)
        self.strategy = self.strategy_cls(self.data_handler, self.events)
        self.portfolio = self.portfolio_cls(self.data_handler, self.events,
                                            self.start_date,
                                            self.initial_capital)
        self.execution_handler = self.execution_handler_cls(self.events, self.portfolio)

In [7]:
if __name__ == "__main__":
    symbol_list = ['BTC_ETH', ]
    initial_capital = 1.0
    heartbeat = 1
    start_date = datetime.now()

    backtest = PoloniexBacktest(
        None,
        symbol_list,
        initial_capital,
        heartbeat,
        start_date,
        PoloniexDataHandler, # for poloniex
        SimulatedExecutionHandler,
        NaivePortfolio,
        PredictStrategy,
        fields=['open', 'high', 'low', 'close', 'volume'],
        verbose=1
    )
    backtest.simulate_trading()

Start backtesting...
Got new event: MarketEvent
Got new event: MarketEvent
Got new event: MarketEvent
Got new event: MarketEvent
Got new event: MarketEvent
Got new event: MarketEvent


KeyboardInterrupt: 